In [ ]:
from __future__ import annotations
import os, requests
from typing import Optional, Dict, Any, Sequence, Annotated, Literal, List
from datetime import datetime, timedelta
from pydantic import BaseModel, Field

# LangChain / LangGraph
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain.tools import tool
from langgraph.graph import StateGraph, END
from langgraph.graph.message import add_messages, AnyMessage
from langgraph.prebuilt import ToolNode
from typing_extensions import TypedDict

from dotenv import load_dotenv

# 품목 검색 모듈 임포트
from llm_product_searcher import LLMProductSearcher

load_dotenv()

# 환경 변수
OPENAI_API_KEY = os.environ.get("OPENAI_API_KEY")
KAMIS_CERT_KEY = os.environ.get("KAMIS_API_KEY")
KAMIS_CERT_ID = os.environ.get("KAMIS_CERT_ID")
DB_PATH = os.getenv("DB_PATH", "kamis_api_list.db")

KAMIS_URL = (
    "http://www.kamis.or.kr/service/price/xml.do?action=recentlyPriceTrendList"
)

# ===========================================
# 품목 검색기 초기화 (전역 인스턴스)
# ===========================================
product_searcher = LLMProductSearcher(DB_PATH)


# ===========================================
# Tool Schema 정의
# ===========================================


class KamisPriceQuery(BaseModel):
    """KAMIS 농축수산물 가격 조회 파라미터"""


    # 여기에 product_searcher.get_name_code_pairs(사용자입력)을 전달 받아서 명시해줘야되는데 어떻게 하지
    p_productno: str = Field(
        description=(
            f"{}\n"
            "이 필드를 기반으로 자동으로 품목 코드를 검색합니다."
        )
    )

    p_regday: str = Field(
        description=(
            "조회 날짜 (YYYY-MM-DD):\n"
            f"- 오늘: {datetime.now().strftime('%Y-%m-%d')}\n"
            f"- 어제: {(datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')}\n"
            "- 미입력시: 가장 최근 데이터 자동 조회\n"
        ),
        default="",
    )

    p_returntype: Literal["json", "xml"] = Field(
        "json", description="응답 형식 (json 또는 xml, 기본값: json)"
    )


# ===========================================
# Tool 구현
# ===========================================


@tool("get_kamis_price_trends", args_schema=KamisPriceQuery)
def get_kamis_price(
    user_query: str,
    p_regday: str = "",
    p_returntype: Literal["json", "xml"] = "json",
) -> Dict[str, Any]:
    """
    KAMIS API를 통해 농축수산물 일별 가격 정보를 조회합니다.

    사용자 질의(user_query)에서 자동으로 품목을 추출하여 품목 코드를 찾습니다.
    여러 품목이 검색되면 모두 조회합니다.
    
    Args:
        user_query: 사용자의 원본 질의 (예: "배추 가격")
        p_regday: 조회 날짜 (선택, 미지정시 최근 데이터)
        p_returntype: 응답 형식 (json/xml)
    
    Returns:
        API 응답 데이터 또는 오류 메시지
    """
    if product_searcher is None:
        return {
            "error": "Product searcher not initialized",
            "message": "품목 검색 모듈을 초기화할 수 없습니다."
        }

    try:
        # 1. 사용자 쿼리에서 품목 코드 검색
        product_pairs = product_searcher.get_name_code_pairs(user_query)
        
        if not product_pairs:
            return {
                "error": "No products found",
                "message": f"'{user_query}'에서 품목을 찾을 수 없습니다.",
                "suggestion": "다른 품목명으로 시도해보세요."
            }

        # 2. 각 품목에 대해 API 호출
        results = []
        for product in product_pairs:
            product_code = product["product_code"]
            product_name = product["product_name"]
            
            params = {
                "action": "recentlyPriceTrendList",
                "p_cert_key": KAMIS_CERT_KEY,
                "p_cert_id": KAMIS_CERT_ID,
                "p_returntype": p_returntype,
                "p_productno": product_code,
            }

            if p_regday:
                params["p_regday"] = p_regday

            try:
                response = requests.get(KAMIS_URL, params=params, timeout=10)
                response.raise_for_status()
                data = response.json()
                
                results.append({
                    "product_code": product_code,
                    "product_name": product_name,
                    "data": data,
                    "status": "success"
                })
            except Exception as e:
                results.append({
                    "product_code": product_code,
                    "product_name": product_name,
                    "error": str(e),
                    "status": "failed"
                })
        
        return {
            "query": user_query,
            "found_products": len(product_pairs),
            "results": results
        }
        
    except Exception as e:
        return {
            "error": str(e),
            "message": "품목 검색 또는 API 호출 중 오류 발생"
        }


# ===========================================
# Agent 구성
# ===========================================


class AgentState(TypedDict):
    messages: Annotated[Sequence[AnyMessage], add_messages]


SYSTEM_PROMPT = f"""KAMIS 농축수산물 가격 정보 조회 시스템

## 입력
농축수산물 가격 추이 관련 질문 (품목, 날짜 포함 가능)

## 처리
1. 질의 분석:
   - 품목 → 품목코드 (쌀: 111, 닭: 9901 등)
   - 날짜 → YYYY-MM-DD (오늘: {datetime.now().strftime('%Y-%m-%d')})

2. get_kamis_price 도구로 조회
   - 소매/도매별 가능 지역 확인 필수
   - 데이터 없으면 날짜 조정 후 재시도

## 출력 형식
```
[조회 조건]
품목명: {{품목명}}:
품목코드: {{품목코드}}
날짜: {{조회날짜}}

[가격 정보]
{{데이터}}

[참고사항]
{{필요시 추가 설명}}

## 주의사항
조회 실패시 오류 원인 명시. 불필요한 인사말, 부연설명 생략."""


def build_kamis_agent():
    """KAMIS 조회 에이전트 생성"""

    llm = ChatOpenAI(
        model="gpt-5-mini",
        temperature=0,
        api_key=OPENAI_API_KEY,
        reasoning_effort="minimal"
    )
    tools = [get_kamis_price]
    llm_with_tools = llm.bind_tools(tools)

    graph = StateGraph(AgentState)

    def agent_node(state: AgentState):
        response = llm_with_tools.invoke(state["messages"])
        return {"messages": [response]}

    tool_node = ToolNode(tools)

    graph.add_node("agent", agent_node)
    graph.add_node("tools", tool_node)
    graph.set_entry_point("agent")

    def should_continue(state: AgentState):
        last_message = state["messages"][-1]
        if isinstance(last_message, AIMessage) and last_message.tool_calls:
            return "tools"
        return END

    graph.add_conditional_edges("agent", should_continue, {"tools": "tools", END: END})
    graph.add_edge("tools", "agent")

    return graph.compile()


# ===========================================
# 사용자 인터페이스
# ===========================================


def query_kamis(user_query: str, verbose: bool = False) -> str:
    """
    KAMIS 가격 조회 실행

    Args:
        user_query: 사용자 질의 (예: "배추 가격 알려줘")
        verbose: True시 전체 메시지 히스토리 출력

    Returns:
        구조화된 가격 정보 텍스트
    """
    app = build_kamis_agent()

    messages = [SystemMessage(content=SYSTEM_PROMPT), HumanMessage(content=user_query)]

    result = app.invoke({"messages": messages})

    if verbose:
        print("=== 전체 대화 히스토리 ===")
        for msg in result["messages"]:
            print(f"\n[{msg.__class__.__name__}]")
            if hasattr(msg, "content"):
                print(msg.content)
            if hasattr(msg, "tool_calls") and msg.tool_calls:
                print(f"Tool Calls: {msg.tool_calls}")
        print("\n" + "=" * 50 + "\n")

    return result["messages"][-1].content


# ===========================================
# 직접 품목 검색 테스트 함수
# ===========================================


def test_product_search(query: str):
    """품목 검색 기능만 단독 테스트"""
    if product_searcher is None:
        print("Error: Product searcher not available")
        return
    
    print(f"\n검색 쿼리: {query}")
    pairs = product_searcher.get_name_code_pairs(query)
    print(f"검색 결과 ({len(pairs)}개):")
    for p in pairs:
        print(f"  - {p['product_name']} (코드: {p['product_code']})")


# ===========================================
# 테스트
# ===========================================

if __name__ == "__main__":
    # 테스트 케이스
    test_queries = "최근 배추 가격 추이를 알려줘"

    print("🌾 KAMIS 농축수산물 가격 조회 에이전트 테스트\n")

    query = test_queries
    print(f"질문: {query}")
    print(f"답변:\n{query_kamis(query)}")
    print("-" * 80)
